In [36]:
import json
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

In [37]:
# -----------------------------
# 1. JSON 파일 읽기
# -----------------------------
with open("../../../measure/ACC/parking_data/parking_nonhangang_final.json", "r", encoding="utf-8") as f:
    nonhangang_data = json.load(f)

with open("../../../measure/ACC/parking_data/parking_hangang.json", "r", encoding="utf-8") as f:
    hangang_data = json.load(f)

In [38]:
# -----------------------------
# 2. GeoDataFrame 불러오기
# -----------------------------
gu = gpd.read_file("../../../map/result/gu.gpkg", layer="gu").to_crs(4326)
parks = gpd.read_file("../../../map/result/parks.gpkg", layer="hangang_parks").to_crs(4326)

In [39]:
# -----------------------------
# 3. 한강 외 공원 → 구별 spaces 총합
# -----------------------------
gu_spaces = {}
for park, info in nonhangang_data.items():
    g = info["구"] 
    try:
        spaces = int(info["spaces"]) if info["spaces"] not in [None, ""] else 0
    except ValueError:
        spaces = 0

    if isinstance(g, list):
        for gg in g:
            gu_spaces.setdefault(gg, []).append(spaces)
    else:
        gu_spaces.setdefault(g, []).append(spaces)

gu_sum = {g: sum(sp) for g, sp in gu_spaces.items()}
gu_df = pd.DataFrame([{"자치구": g, "spaces": t} for g, t in gu_sum.items()])
gu = gu.merge(gu_df, left_on="SIGUNGU_NM", right_on="자치구", how="left").drop(columns=["자치구"])


In [40]:
# -----------------------------
# 4. 한강공원 → 개별 spaces
# -----------------------------
park_df = pd.DataFrame([
    {"공원명": info["공원명"], "spaces": int(info["spaces"]) if info["spaces"] not in [None, ""] else 0}
    for _, info in hangang_data.items()
])
park_name_map = {
    "Banpo": "반포한강공원", "Gangseo": "강서한강공원", "Gwangnaru": "광나루한강공원",
    "Ichon": "이촌한강공원", "Jamsil": "잠실한강공원", "Jamwon": "잠원한강공원",
    "Mangwon": "망원한강공원", "Nanji": "난지한강공원", "Ttukseom": "뚝섬한강공원",
    "Yanghwa": "양화한강공원", "Yeouido": "여의도한강공원"
}
parks["park_name"] = parks["park_name"].map(park_name_map)
parks = parks.merge(park_df, left_on="park_name", right_on="공원명", how="left").drop(columns=["공원명"])

In [41]:
parks

,park_id,park_name,gu_name,id,geometry,spaces
0,HP11,강서한강공원,강서구,5,"MULTIPOLYGON (((126.80426 37.59206, 126.81101 ...",52
1,HP09,난지한강공원,마포구,6,"MULTIPOLYGON (((126.88433 37.55432, 126.88411 ...",618
2,HP10,망원한강공원,마포구,7,"MULTIPOLYGON (((126.8927 37.55844, 126.8942 37...",332
3,HP08,양화한강공원,영등포구,8,"MULTIPOLYGON (((126.9064 37.53549, 126.9047 37...",501
4,HP07,여의도한강공원,영등포구,9,"MULTIPOLYGON (((126.90823 37.53402, 126.91426 ...",1782
5,HP05,반포한강공원,서초구,10,"MULTIPOLYGON (((126.98041 37.50654, 126.98125 ...",819
6,HP04,잠원한강공원,서초구,11,"MULTIPOLYGON (((126.99992 37.51894, 127.00099 ...",722
7,HP04,잠원한강공원,강남구,12,"MULTIPOLYGON (((127.00858 37.5256, 127.01001 3...",722
8,HP06,이촌한강공원,용산구,13,"MULTIPOLYGON (((126.95203 37.53037, 126.95279 ...",371
9,HP03,뚝섬한강공원,광진구,1,"MULTIPOLYGON (((127.0592 37.5337, 127.06671 37...",674


In [42]:
from matplotlib.colors import LogNorm

# -----------------------------
# 5. colormap (구 + 공원 합친 전체 기준, 로그 스케일)
# -----------------------------
all_min = 1   # 0은 로그 스케일에서 불가능하니 최소 1로 설정
all_max = max(gu["spaces"].max(), parks["spaces"].max())

cmap = plt.cm.Purples
norm = LogNorm(vmin=all_min, vmax=all_max)


In [43]:
# -----------------------------
# 6. folium 지도 생성
# -----------------------------
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, tiles="CartoDB positron")

# 자치구 레이어
folium.GeoJson(
    gu,
    style_function=lambda f: {
        "fillColor": (
            "#eeeeee" if pd.isna(f["properties"]["spaces"]) or f["properties"]["spaces"] == 0
            else colors.rgb2hex(cmap(norm(f["properties"]["spaces"])))
        ),
        "color": "#666",
        "weight": 1,
        "fillOpacity": 0 if f["properties"]["spaces"] == 0 else 0.8  # 0이면 투명
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["SIGUNGU_NM", "spaces"],
        aliases=["자치구", "주차면수 합계"]
    )
).add_to(m)

# 한강공원 레이어
folium.GeoJson(
    parks,
    name="한강공원",
    style_function=lambda f: {
        "fillColor": (
            "#eeeeee" if pd.isna(f["properties"]["spaces"]) or f["properties"]["spaces"] == 0
            else colors.rgb2hex(cmap(norm(f["properties"]["spaces"])))
        ),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 0 if f["properties"]["spaces"] == 0 else 0.8  # 0이면 투명
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["park_name", "spaces"],
        aliases=["공원", "주차면수"]
    )
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

In [44]:
# 저장
m.save("../../result/ACC/parking_spaces_visualization.html")
print("parking_spaces_visualization.html 저장 완료")

parking_spaces_visualization.html 저장 완료
